## Data 한줄씩 읽어서 Streaming으로 흘려보내기

In [1]:
from kafka import KafkaProducer, KafkaConsumer

import logging
from json import dumps, loads
import csv
import time

logging.basicConfig(level=logging.INFO)

In [2]:
producer=KafkaProducer(
            bootstrap_servers=['localhost:9092'],
            value_serializer=lambda x: dumps(x).encode('utf-8')
        )

INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: connecting to localhost:9092 [('127.0.0.1', 9092) IPv4]
INFO:kafka.conn:Probing node bootstrap-0 broker version
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: Connection complete.
INFO:kafka.conn:Broker version identified as 2.5.0
INFO:kafka.conn:Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup


In [3]:
consumer=KafkaConsumer("covid_1", #읽어올 토픽의 이름 필요.
                        bootstrap_servers=['localhost:9092'], # 어떤 서버에서 읽어 올지 지정
                        auto_offset_reset="earliest", # 어디서부터 값을 읽어올지 (earlest 가장 처음 latest는 가장 최근)
                        enable_auto_commit=True, # 완료되었을 떄 문자 전송
                        group_id='my-group', # 그룹핑하여 토픽 지정할 수 있다 > 같은 컨슈머로 작업
                        value_deserializer=lambda x: loads(x.decode('utf-8')), # 역직렬화 ( 받을 떄 ) ; 메모리에서 읽어오므로 loads라는 함수를 이용한다. // 직렬화 (보낼 떄)
                        consumer_timeout_ms=1000
)

INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: connecting to localhost:9092 [('127.0.0.1', 9092) IPv4]
INFO:kafka.conn:Probing node bootstrap-0 broker version
INFO:kafka.conn:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv4 ('127.0.0.1', 9092)]>: Connection complete.
INFO:kafka.conn:Broker version identified as 2.5.0
INFO:kafka.conn:Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
INFO:kafka.consumer.subscription_state:Updating subscribed topics to: ('covid_1',)


In [20]:
with open('./november_2021_COVID-19_Twitter_Streaming_Dataset.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    
    for message in reader:

        message = int(message[0])
        producer.send('covid_1', value=message)
        producer.flush()
        
        for message in consumer:

            message_t = message.topic
            message_p = message.partition
            message_o = message.offset
            message_k = message.key
            message_v = message.value

        print(message_v)
        break
        
print('Done')

1454961019550265345
Done
